# Make a nwb file

Let's create an nwb file! To do this, you'll need a metadata yaml file, and all your data. 

The following python packages need to be installed:

- spikeinterface
- neuroconv (Chris style: https://github.com/chrishalcrow/neuroconv/tree/add_time_series)

First step: let's read in the data and couple the recording to a probe.

In [2]:
import numpy as np
import probeinterface as pi


def tetrode_probe():

    geometry = np.array([[   0,  0], [  25,  0], [  25, 25], [  0,  25],
                         [200, 200], [225, 200], [225, 225], [200, 225],
                         [400, 400], [425, 400], [425, 425], [400, 425],
                         [600, 600], [625, 600], [625, 625], [600, 625]])
    
    probe = pi.Probe(ndim=2, si_units='um')
    probe.set_contacts(positions=geometry, shapes='circle', shape_params={'radius': 5})
    probe.set_device_channel_indices(np.arange(16))
    probe.set_shank_ids(shank_ids=[0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3])

    return probe


In [35]:
from neuroconv.datainterfaces import OpenEphysRecordingInterface
from neuroconv.utils.dict import load_dict_from_file
from neuroconv.datainterfaces import CsvTimeSeriesInterface

from neuroconv.datainterfaces import OpenEphysRecordingInterface
folder_path = f"data"
my_ephys_interface = OpenEphysRecordingInterface(folder_path=folder_path, stream_name="Signals CH")

probe = tetrode_probe()
my_ephys_interface.recording_extractor.set_probe(probe, group_mode="by_shank", in_place=True)

ephys_folder_path = f"data"
metadata_path = f"metadata_with_pos.yml"

# read in the metadata
metadata_from_yaml = load_dict_from_file(file_path=metadata_path)

# get the ephys data
my_ephys_interface = OpenEphysRecordingInterface(folder_path=ephys_folder_path, stream_name="Signals CH")
probe = tetrode_probe()
my_ephys_interface.recording_extractor.set_probe(probe, group_mode="by_shank", in_place=True)

# get the time series data. Note the series_names MUST match the time_series_names in your yaml file,
# so that the metadata and data get coupled.
file_path = f"data/position_data.csv"
ts_interface = CsvTimeSeriesInterface(file_path=file_path, series_name = 'Position')
file_path2 = f"foo3.csv"
ts_interface2 = CsvTimeSeriesInterface(file_path=file_path2, series_name = 'Eye-tracking')

Two possible problems: 1) Sometimes you might have saved more data in your TimeSeries that you really want. 2) The time column should be called 'timestamps', for internal use in Neuroconv.

Consider the time series we've just loaded in:

In [36]:
ts_interface.dataframe

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,x_position_cm,trial_number,trial_type,time_seconds
0,0,0,0,0,7.650576,1,0,0.000
1,1,1,1,30,7.649772,1,0,0.001
2,2,2,2,60,7.648908,1,0,0.002
3,3,3,3,90,7.648025,1,0,0.003
4,4,4,4,120,7.647167,1,0,0.004
...,...,...,...,...,...,...,...,...
1901700,1901700,1901700,1901700,57051000,89.375000,197,0,1901.700
1901701,1901701,1901701,1901701,57051030,89.375000,197,0,1901.701
1901702,1901702,1901702,1901702,57051060,89.375000,197,0,1901.702
1901703,1901703,1901703,1901703,57051090,89.375000,197,0,1901.703


Let's only keep the time, position, trial number and trial type columns. Then rename 'time_seconds' to 'timestamps'.

In [37]:
import pandas as pd

ts_interface.dataframe = ts_interface.dataframe[['time_seconds', 'x_position_cm', 'trial_number', 'trial_type']]
ts_interface.dataframe = new_ts_interface.dataframe.rename(columns={"time_seconds": 'timestamps'})


NWB is a bit particular about how times and dates are stored. So let's set that carefully...

In [38]:
from datetime import datetime
from dateutil import tz

session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata_from_yaml['NWBFile']['session_start_time'] = session_start_time
metadata_from_yaml['Ecephys']['ElectricalSeries']['starting_time'] = session_start_time


Combine all the interfaces into one "pipe"

In [ ]:
from neuroconv import ConverterPipe
data_interfaces = [my_ephys_interface, ts_interface, ts_interface2]
all_the_data = ConverterPipe(data_interfaces=data_interfaces)

And we can now convert it!

nwbfile_path = "harrys_lovely_daya.nwb"
all_the_data.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata_from_yaml)

Well done!

# How to use with SpikeInterface

One reason for doing this is that everything should be set up so that importing this into spikeinterface is SUPER simple. Let's try, for fun.

In [ ]:
from spikeinterface.extractors import read_nwb

recording = read_nwb("harrys_lovely_daya.nwb")
print(recording)

Cool! And you'll notice that the probe is attached too (although it claims it isn't)

In [ ]:
print(recording.has_probe())
print(recording.get_channel_locations())

This means we can do spike sorting immediately, without faffing and worrying about probes.

In [ ]:
import spikeinterface.full as si
si.run_sorter(sorter_name="simple", recording=recording, output_folder="simple")

Which is nice and easy for everyone :)